In [123]:
import inspect
import time
import collections
import math
import os
import random
import glob
import nltk
import pandas as pd
import io
import re
from tqdm import tqdm
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [124]:
DATA_PATH = "/kaggle/dev/ashish/word_vectors_harry_potter/data/"
SAVE_PATH = "/kaggle/dev/ashish/word_vectors_harry_potter/trained_model"

In [125]:
files = sorted(glob.glob(DATA_PATH + "*.txt"))

for file in files:
    print(os.path.basename(file))

hp1.txt
hp2.txt
hp3.txt
hp4.txt
hp5.txt
hp6.txt
hp7.txt


In [126]:
text_raw = u""
for file in files:
    with io.open(file,'r',encoding='utf8') as f:
        text_raw += f.read()
text_raw = text_raw.replace('\n', ' ')
print('Books contain {} characters'.format(len(text_raw)))

Books contain 6286105 characters


In [127]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
sentences_raw = sent_detector.tokenize(text_raw)
print('Books contain {} sentences'.format(len(sentences_raw)))
print(sentences_raw[0:10])

Books contain 61595 sentences
['Harry Potter  and the Sorcerer’s Stone      by  J. K. Rowling  Illustrations by Mary Grandpré    Arthur A. Levine Books  An Imprint of Scholastic Press.', 'For Jessica, who loves stories  for Anne, who loved them too;  and for Di, who heard this one first.', 'Text copyright © 1997 by J.K. Rowling  Illustrations by Mary GrandPré copyright © 1998 Warner Bros.  All rights reserved.', 'Published by Scholastic Press, a division of Scholastic Inc.,  Publishers since 1920  SCHOLASTIC, SCHOLASTIC PRESS, and the LANTERN LOGO  are trademarks and/or registered trademarks of Scholastic Inc.    HARRY POTTER and all related characters and elements are trademarks of Warner Bros.    No part of this publication may be reproduced, or stored in a retrieval system, or transmitted in any form or by any means, electronic, mechanical, photocopying, recording, or otherwise, without written permission of the publisher.', 'For information regarding permissions, write to Scholasti

In [128]:
def clean_sentences(sentence):
    s = re.sub(r'[^a-zA-Z]', " ", sentence)
    s = s.split()
    return s

words = []
for sentence in sentences_raw:
    for word in clean_sentences(sentence):
        words.append(word)
    
print(words[1000:1050])

['the', 'window', 'in', 'his', 'office', 'on', 'the', 'ninth', 'floor', 'If', 'he', 'hadn', 't', 'he', 'might', 'have', 'found', 'it', 'harder', 'to', 'concentrate', 'on', 'drills', 'that', 'morning', 'He', 'didn', 't', 'see', 'the', 'owls', 'swooping', 'past', 'in', 'broad', 'daylight', 'though', 'people', 'down', 'in', 'the', 'street', 'did', 'they', 'pointed', 'and', 'gazed', 'open', 'mouthed', 'as']


In [129]:
word_counts = collections.Counter(words)
vocabulary_inv = [x[0] for x in word_counts.most_common()]
vocabulary = {word: idx for idx, word in enumerate(vocabulary_inv)}
VOCAB_SIZE = len(vocabulary)
print('vocabulary_inv', len(vocabulary_inv))
print('vocabulary', len(vocabulary))
print('vocab size', vocab_size)

vocabulary_inv 24986
vocabulary 24986
vocab size 24986


In [130]:
tensor = np.asarray(list(map(vocabulary.get, words)))
np.save(DATA_PATH + 'tensor', tensor)
print(tensor.shape)

(1124921,)


In [131]:
BATCH_SIZE = 50
SEQ_LENGTH = 25

def create_batches(batch_size, seq_length, tensor):
    num_batches = int(tensor.size / (batch_size * seq_length))
    if num_batches==0:
        assert False, "Not enough data. Make seq_length and batch_size small."

    tensor = tensor[:num_batches * batch_size * seq_length]
    xdata = tensor
    ydata = np.copy(tensor)

    ydata[:-1] = xdata[1:]
    ydata[-1] = xdata[0]
    x_batches = np.split(xdata.reshape(batch_size, -1), num_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), num_batches, 1)
    return x_batches, y_batches

x_batches, y_batches = create_batches(BATCH_SIZE, SEQ_LENGTH, tensor)
print('len(x_batches)', len(x_batches))
print('len(y_batches)', len(y_batches))
print('x_batches[0]', x_batches[0])
print('y_batches[0]', y_batches[0])

len(x_batches) 899
len(y_batches) 899
x_batches[0] [[   5  136    2 ..., 2777 5642  478]
 [  21    0 6199 ..., 1456  372    1]
 [1242   26 1099 ...,    2 5858   58]
 ..., 
 [3235   60   20 ...,  115    6  561]
 [ 562  142   46 ...,  247  160 1266]
 [  59  211   22 ...,   22  772   19]]
y_batches[0] [[  136     2     0 ...,  5642   478 11512]
 [    0  6199   486 ...,   372     1   938]
 [   26  1099    31 ...,  5858    58   471]
 ..., 
 [   60    20    83 ...,     6   561    80]
 [  142    46    81 ...,   160  1266     9]
 [  211    22  1168 ...,   772    19   474]]


In [143]:
# Define model params
NUM_LAYERS = 2
RNN_SIZE = 256
NUM_EPOCHS = 1
GRAD_CLIP = 5
LEARNING_RATE = 0.002
DECAY_RATE = 0.666

In [144]:
# Model
graph = tf.Graph()
with graph.as_default():
    input_data = tf.placeholder(tf.int32, shape=[BATCH_SIZE, SEQ_LENGTH], name='input_data')
    targets = tf.placeholder(tf.int32, shape=[BATCH_SIZE, SEQ_LENGTH], name='targets')
    epoch_pointer = tf.Variable(0, name="epoch_pointer", trainable=False)
    cells = []
    for _ in range(NUM_LAYERS):
        cell = tf.contrib.rnn.BasicLSTMCell(RNN_SIZE)
        cells.append(cell)

    rnn_cell = tf.contrib.rnn.MultiRNNCell(cells)
    initial_state = rnn_cell.zero_state(BATCH_SIZE, tf.float32)
    
    with tf.variable_scope('rnnlm'):
        softmax_w = tf.get_variable('softmax_w', [RNN_SIZE, VOCAB_SIZE])
        softmax_b = tf.get_variable('softmax_b', [VOCAB_SIZE])
        embedding = tf.get_variable('embedding', [VOCAB_SIZE, RNN_SIZE])
        inputs = tf.split(tf.nn.embedding_lookup(embedding, input_data), SEQ_LENGTH, 1)
        inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
        
    outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, initial_state, rnn_cell, loop_function=None, scope='rnnlm')
    output = tf.reshape(tf.concat(outputs, 1), [-1, RNN_SIZE])
    logits = tf.matmul(output, softmax_w) + softmax_b
    probs = tf.nn.softmax(logits)
    loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], 
                                                             [tf.reshape(targets, [-1])],
                                                             [tf.ones([BATCH_SIZE * SEQ_LENGTH])],
                                                             VOCAB_SIZE)
    cost = tf.reduce_sum(loss) / (BATCH_SIZE * SEQ_LENGTH)
    final_state = last_state
    lr = tf.Variable(0.0, trainable=False)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), GRAD_CLIP)
    optimizer = tf.train.AdamOptimizer(lr)
    train_op = optimizer.apply_gradients(zip(grads, tvars))
    

In [141]:
NUM_BATCHES = len(x_batches)
with tf.Session(graph=graph) as sess:
    #train_writer.add_graph(sess.graph)
    tf.global_variables_initializer().run()
    #saver = tf.train.Saver(tf.global_variables())
    # restore model
#     if args.init_from is not None:
#         saver.restore(sess, ckpt.model_checkpoint_path)
    for e in range(epoch_pointer.eval(), NUM_EPOCHS):
        sess.run(tf.assign(lr, LEARNING_RATE * (DECAY_RATE ** e)))
        batch_pointer = 0
        state = sess.run(initial_state)
        #speed = 0
#         if args.init_from is None:
        assign_op = epoch_pointer.assign(e)
        sess.run(assign_op)
#         if args.init_from is not None:
#             data_loader.pointer = model.batch_pointer.eval()
#             args.init_from = None
        for b in range(batch_pointer, NUM_BATCHES):
            start = time.time()
            x, y = x_batches[b], y_batches[b]
            feed = {input_data: x, targets: y, initial_state: state}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)
            #train_writer.add_summary(summary, e * data_loader.num_batches + b)
            #speed = time.time() - start
            #if (e * NUM_BATCHES + b) % BATCH_SIZE == 0:
            print("{}/{} (epoch {}), train_loss = {:.3f}" \
                .format(e * NUM_BATCHES + b,
                        NUM_EPOCHS * NUM_BATCHES,
                        e, train_loss))
#             if (e * data_loader.num_batches + b) % args.save_every == 0 \
#                     or (e==args.num_epochs-1 and b == data_loader.num_batches-1): # save for the last result
#                 checkpoint_path = os.path.join(args.save_dir, 'model.ckpt')
#                 saver.save(sess, checkpoint_path, global_step = e * data_loader.num_batches + b)
#                 print("model saved to {}".format(checkpoint_path))
    #train_writer.close()

0/44950 (epoch 0), train_loss = 10.125
1/44950 (epoch 0), train_loss = 10.122
2/44950 (epoch 0), train_loss = 10.115
3/44950 (epoch 0), train_loss = 10.074
4/44950 (epoch 0), train_loss = 9.769
5/44950 (epoch 0), train_loss = 9.171
6/44950 (epoch 0), train_loss = 8.637
7/44950 (epoch 0), train_loss = 8.118
8/44950 (epoch 0), train_loss = 7.749
9/44950 (epoch 0), train_loss = 7.485
10/44950 (epoch 0), train_loss = 7.226
11/44950 (epoch 0), train_loss = 7.315
12/44950 (epoch 0), train_loss = 7.317
13/44950 (epoch 0), train_loss = 7.199
14/44950 (epoch 0), train_loss = 7.419
15/44950 (epoch 0), train_loss = 7.367
16/44950 (epoch 0), train_loss = 7.498
17/44950 (epoch 0), train_loss = 7.561
18/44950 (epoch 0), train_loss = 7.579
19/44950 (epoch 0), train_loss = 7.532
20/44950 (epoch 0), train_loss = 7.225
21/44950 (epoch 0), train_loss = 7.603
22/44950 (epoch 0), train_loss = 7.381
23/44950 (epoch 0), train_loss = 7.338
24/44950 (epoch 0), train_loss = 7.456
25/44950 (epoch 0), train_loss 

209/44950 (epoch 0), train_loss = 7.262
210/44950 (epoch 0), train_loss = 7.091
211/44950 (epoch 0), train_loss = 7.215
212/44950 (epoch 0), train_loss = 7.112
213/44950 (epoch 0), train_loss = 7.196
214/44950 (epoch 0), train_loss = 7.059
215/44950 (epoch 0), train_loss = 7.055
216/44950 (epoch 0), train_loss = 7.127
217/44950 (epoch 0), train_loss = 7.094
218/44950 (epoch 0), train_loss = 7.059
219/44950 (epoch 0), train_loss = 7.135
220/44950 (epoch 0), train_loss = 7.120
221/44950 (epoch 0), train_loss = 7.064
222/44950 (epoch 0), train_loss = 7.119
223/44950 (epoch 0), train_loss = 7.282
224/44950 (epoch 0), train_loss = 7.016
225/44950 (epoch 0), train_loss = 7.121
226/44950 (epoch 0), train_loss = 7.141
227/44950 (epoch 0), train_loss = 7.103
228/44950 (epoch 0), train_loss = 7.144
229/44950 (epoch 0), train_loss = 7.115
230/44950 (epoch 0), train_loss = 7.175
231/44950 (epoch 0), train_loss = 7.112
232/44950 (epoch 0), train_loss = 7.116
233/44950 (epoch 0), train_loss = 7.158


416/44950 (epoch 0), train_loss = 7.112
417/44950 (epoch 0), train_loss = 7.101
418/44950 (epoch 0), train_loss = 7.126
419/44950 (epoch 0), train_loss = 7.192
420/44950 (epoch 0), train_loss = 7.113
421/44950 (epoch 0), train_loss = 7.045
422/44950 (epoch 0), train_loss = 7.211
423/44950 (epoch 0), train_loss = 7.128
424/44950 (epoch 0), train_loss = 7.065
425/44950 (epoch 0), train_loss = 7.077
426/44950 (epoch 0), train_loss = 6.983
427/44950 (epoch 0), train_loss = 7.110
428/44950 (epoch 0), train_loss = 7.076
429/44950 (epoch 0), train_loss = 7.138
430/44950 (epoch 0), train_loss = 7.224
431/44950 (epoch 0), train_loss = 7.069
432/44950 (epoch 0), train_loss = 7.267
433/44950 (epoch 0), train_loss = 7.068
434/44950 (epoch 0), train_loss = 7.074
435/44950 (epoch 0), train_loss = 7.120
436/44950 (epoch 0), train_loss = 7.010
437/44950 (epoch 0), train_loss = 7.163
438/44950 (epoch 0), train_loss = 7.076
439/44950 (epoch 0), train_loss = 7.178
440/44950 (epoch 0), train_loss = 7.064


623/44950 (epoch 0), train_loss = 6.979
624/44950 (epoch 0), train_loss = 7.337
625/44950 (epoch 0), train_loss = 7.272
626/44950 (epoch 0), train_loss = 7.092
627/44950 (epoch 0), train_loss = 6.902
628/44950 (epoch 0), train_loss = 7.072
629/44950 (epoch 0), train_loss = 7.093
630/44950 (epoch 0), train_loss = 6.989
631/44950 (epoch 0), train_loss = 7.126
632/44950 (epoch 0), train_loss = 7.160
633/44950 (epoch 0), train_loss = 7.118
634/44950 (epoch 0), train_loss = 6.983
635/44950 (epoch 0), train_loss = 7.085
636/44950 (epoch 0), train_loss = 7.130
637/44950 (epoch 0), train_loss = 7.148
638/44950 (epoch 0), train_loss = 7.132
639/44950 (epoch 0), train_loss = 7.273
640/44950 (epoch 0), train_loss = 7.129
641/44950 (epoch 0), train_loss = 7.130
642/44950 (epoch 0), train_loss = 7.164
643/44950 (epoch 0), train_loss = 7.089
644/44950 (epoch 0), train_loss = 7.064
645/44950 (epoch 0), train_loss = 7.269
646/44950 (epoch 0), train_loss = 7.233
647/44950 (epoch 0), train_loss = 6.934


829/44950 (epoch 0), train_loss = 7.239
830/44950 (epoch 0), train_loss = 6.988
831/44950 (epoch 0), train_loss = 6.944
832/44950 (epoch 0), train_loss = 7.051
833/44950 (epoch 0), train_loss = 7.068
834/44950 (epoch 0), train_loss = 7.076
835/44950 (epoch 0), train_loss = 7.015
836/44950 (epoch 0), train_loss = 7.175
837/44950 (epoch 0), train_loss = 7.159
838/44950 (epoch 0), train_loss = 7.018
839/44950 (epoch 0), train_loss = 7.144
840/44950 (epoch 0), train_loss = 7.110
841/44950 (epoch 0), train_loss = 7.173
842/44950 (epoch 0), train_loss = 7.096
843/44950 (epoch 0), train_loss = 7.205
844/44950 (epoch 0), train_loss = 7.313
845/44950 (epoch 0), train_loss = 7.244
846/44950 (epoch 0), train_loss = 7.147
847/44950 (epoch 0), train_loss = 7.128
848/44950 (epoch 0), train_loss = 7.185
849/44950 (epoch 0), train_loss = 7.327
850/44950 (epoch 0), train_loss = 7.189
851/44950 (epoch 0), train_loss = 7.396
852/44950 (epoch 0), train_loss = 7.293
853/44950 (epoch 0), train_loss = 7.121


...

44050/44950 (epoch 48), train_loss = 5.742
44051/44950 (epoch 49), train_loss = 5.899
44052/44950 (epoch 49), train_loss = 5.769
44053/44950 (epoch 49), train_loss = 5.653
44054/44950 (epoch 49), train_loss = 5.758
44055/44950 (epoch 49), train_loss = 5.743
44056/44950 (epoch 49), train_loss = 5.745
44057/44950 (epoch 49), train_loss = 5.709
44058/44950 (epoch 49), train_loss = 5.817
44059/44950 (epoch 49), train_loss = 5.852
44060/44950 (epoch 49), train_loss = 5.742
44061/44950 (epoch 49), train_loss = 5.704
44062/44950 (epoch 49), train_loss = 5.736
44063/44950 (epoch 49), train_loss = 5.826
44064/44950 (epoch 49), train_loss = 5.718
44065/44950 (epoch 49), train_loss = 5.697
44066/44950 (epoch 49), train_loss = 5.645
44067/44950 (epoch 49), train_loss = 5.703
44068/44950 (epoch 49), train_loss = 5.793
44069/44950 (epoch 49), train_loss = 5.723
44070/44950 (epoch 49), train_loss = 5.772
44071/44950 (epoch 49), train_loss = 5.494
44072/44950 (epoch 49), train_loss = 5.871
44073/44950

44241/44950 (epoch 49), train_loss = 5.769
44242/44950 (epoch 49), train_loss = 5.628
44243/44950 (epoch 49), train_loss = 5.384
44244/44950 (epoch 49), train_loss = 5.766
44245/44950 (epoch 49), train_loss = 5.770
44246/44950 (epoch 49), train_loss = 5.724
44247/44950 (epoch 49), train_loss = 5.768
44248/44950 (epoch 49), train_loss = 5.677
44249/44950 (epoch 49), train_loss = 5.706
44250/44950 (epoch 49), train_loss = 5.847
44251/44950 (epoch 49), train_loss = 5.649
44252/44950 (epoch 49), train_loss = 5.698
44253/44950 (epoch 49), train_loss = 5.797
44254/44950 (epoch 49), train_loss = 5.663
44255/44950 (epoch 49), train_loss = 5.797
44256/44950 (epoch 49), train_loss = 5.696
44257/44950 (epoch 49), train_loss = 5.683
44258/44950 (epoch 49), train_loss = 5.760
44259/44950 (epoch 49), train_loss = 5.631
44260/44950 (epoch 49), train_loss = 5.845
44261/44950 (epoch 49), train_loss = 5.686
44262/44950 (epoch 49), train_loss = 5.830
44263/44950 (epoch 49), train_loss = 5.693
44264/44950

44434/44950 (epoch 49), train_loss = 5.625
44435/44950 (epoch 49), train_loss = 5.756
44436/44950 (epoch 49), train_loss = 5.750
44437/44950 (epoch 49), train_loss = 5.587
44438/44950 (epoch 49), train_loss = 5.607
44439/44950 (epoch 49), train_loss = 5.608
44440/44950 (epoch 49), train_loss = 5.625
44441/44950 (epoch 49), train_loss = 5.790
44442/44950 (epoch 49), train_loss = 5.652
44443/44950 (epoch 49), train_loss = 5.704
44444/44950 (epoch 49), train_loss = 5.583
44445/44950 (epoch 49), train_loss = 5.895
44446/44950 (epoch 49), train_loss = 5.755
44447/44950 (epoch 49), train_loss = 5.793
44448/44950 (epoch 49), train_loss = 5.695
44449/44950 (epoch 49), train_loss = 5.622
44450/44950 (epoch 49), train_loss = 5.617
44451/44950 (epoch 49), train_loss = 5.567
44452/44950 (epoch 49), train_loss = 5.664
44453/44950 (epoch 49), train_loss = 5.741
44454/44950 (epoch 49), train_loss = 5.758
44455/44950 (epoch 49), train_loss = 5.766
44456/44950 (epoch 49), train_loss = 5.671
44457/44950

44626/44950 (epoch 49), train_loss = 5.744
44627/44950 (epoch 49), train_loss = 5.693
44628/44950 (epoch 49), train_loss = 5.889
44629/44950 (epoch 49), train_loss = 5.659
44630/44950 (epoch 49), train_loss = 5.808
44631/44950 (epoch 49), train_loss = 5.731
44632/44950 (epoch 49), train_loss = 5.854
44633/44950 (epoch 49), train_loss = 5.841
44634/44950 (epoch 49), train_loss = 5.675
44635/44950 (epoch 49), train_loss = 5.763
44636/44950 (epoch 49), train_loss = 5.847
44637/44950 (epoch 49), train_loss = 5.854
44638/44950 (epoch 49), train_loss = 5.795
44639/44950 (epoch 49), train_loss = 5.765
44640/44950 (epoch 49), train_loss = 5.745
44641/44950 (epoch 49), train_loss = 5.861
44642/44950 (epoch 49), train_loss = 5.809
44643/44950 (epoch 49), train_loss = 5.664
44644/44950 (epoch 49), train_loss = 5.704
44645/44950 (epoch 49), train_loss = 5.705
44646/44950 (epoch 49), train_loss = 5.884
44647/44950 (epoch 49), train_loss = 5.745
44648/44950 (epoch 49), train_loss = 5.759
44649/44950

44817/44950 (epoch 49), train_loss = 5.718
44818/44950 (epoch 49), train_loss = 5.676
44819/44950 (epoch 49), train_loss = 5.700
44820/44950 (epoch 49), train_loss = 5.722
44821/44950 (epoch 49), train_loss = 5.917
44822/44950 (epoch 49), train_loss = 5.745
44823/44950 (epoch 49), train_loss = 5.739
44824/44950 (epoch 49), train_loss = 5.652
44825/44950 (epoch 49), train_loss = 5.654
44826/44950 (epoch 49), train_loss = 5.699
44827/44950 (epoch 49), train_loss = 5.781
44828/44950 (epoch 49), train_loss = 5.563
44829/44950 (epoch 49), train_loss = 5.837
44830/44950 (epoch 49), train_loss = 5.604
44831/44950 (epoch 49), train_loss = 5.548
44832/44950 (epoch 49), train_loss = 5.751
44833/44950 (epoch 49), train_loss = 5.710
44834/44950 (epoch 49), train_loss = 5.673
44835/44950 (epoch 49), train_loss = 5.557
44836/44950 (epoch 49), train_loss = 5.723
44837/44950 (epoch 49), train_loss = 5.619
44838/44950 (epoch 49), train_loss = 5.790
44839/44950 (epoch 49), train_loss = 5.808
44840/44950